In [81]:
from bs4 import BeautifulSoup
import requests
import re

## Zastosowania języka Python w analizie ekonomicznej
Kacper Staroń, 29601
Laboratoria, grupa 2
### Temat 3 - Liczenie odległości miedzy słowami na Wikipedii.
Na potrzeby zadania zaliczeniowego przyjęto że:
 - projekt jest przewidziany do wyszukiwania haseł na angielskiej wikipedii i nie uwzględnia deklinacji
 - na potrzeby liczenia odległości między słowami uwzględniane są jedynie merytoryczne elementy artykułu (paragrafy, sekcje i tabele z selektora CSS #bodyContent), nie całej strony 


In [82]:
def calculateDistance(startWord, searchWord, searchTables = "infobox_only", searchSectionNames = True, searchReferences = False, linksFromTables = "infobox_only"):
    '''
    Główna funkcja
    Argumenty:
        startWord - wyjściowe słowo, hasło ze stroną na wikipedii
        searchWord - słowo dla którego będziemy liczyć odległość od hasła z wikipedii (startWord)
        searchTables - przyjmuje wartości:
            yes - uwzględnij w wyszukiwaniu słowa we wszystkich tabelach, nie tylko zawarte w paragrafach
            infobox_only - uwzględnij w wyszukiwaniu słowa w tabelach klasy "infobox" (tabele po prawej stronie artykułu)
            no - nie uwzględniaj tabel w wyszukiwaniu słów (każdy string inny niż "yes" lub "infobox" będzie traktowany jako "no")
        searchSectionNames - przyjmuje wartości True/False, decyduje czy uwzględnić w wyszukiwaniu słów nazwy Sekcji artykułu
        searchReferences - przyjmuje wartości True/False, decyduje czy uwzględnić w wyszukiwaniu słów sekcję źródeł
        linksFromTables - przyjmuje wartości:
            yes - uwzględnij w wyszukiwaniu linki we wszystkich tabelach, nie tylko zawarte w paragrafach
            infobox_only - uwzględnij w wyszukiwaniu linki w tabelach klasy "infobox" (tabele po prawej stronie artykułu)
            no - nie uwzględniaj tabel w wyszukiwaniu linków (każdy string inny niż "yes" lub "infobox" będzie traktowany jako "no")
    '''

    if searchTables not in ["yes", "no", "infobox_only"]:
        print("Invalid searchTables")
        return -1
    if linksFromTables not in ["yes", "no", "infobox_only"]:
        print("Invalid linksFromTables")
        return -1
    if not isinstance(searchSectionNames, bool):
        print("Invalid searchSectionNames")
        return -1
    if not isinstance(searchReferences, bool):
        print("Invalid searchReferences")
        return -1
    
    try:
        startWord = startWord.strip()
    except: print("Invalid startWord")
    try:
        searchWord = searchWord.strip().lower()
    except: print("invalid searchWord")

    
    links = {parseWordToWikipediaLink(startWord) : 1} #linki i ich odległości od startowego hasła trzymamy w słowniku dla ominięcia duplikatów
    iter = 0

    
    if not validateStartWord(startWord): #sprawdzenie, czy dla podanego startWord istnieje strona na wikipedii
        return "No wikipedia page for provided word" 

    if startWord.lower() == searchWord: #przyjęto, że odległość startowego słowa od samego siebie wynosi 0
        return 0

    

    while True:
        distance = search(iter, links, searchWord, searchTables, searchSectionNames, searchReferences, linksFromTables)
        if distance:
            break
        else:
            iter +=1

    return distance

In [83]:
def validateStartWord(word):
    '''
    Funkcja do walidowania czy dla podanego słowa istnieje hasło na wikipedii
    '''
    resp = requests.get(parseWordToWikipediaLink(word))
    page = resp.text
    soup = BeautifulSoup(page, 'html.parser')
    return len(soup.css.select(".noarticletext")) == 0 #selektor css dla standardowego obiektu wyświetlanego na stronie bez hasła


def parseWordToWikipediaLink(word):
    '''
    Funkcja do zamiany wyrażeń w języku naturalnym na linki do haseł na wikipedii, obsługuje wyrażenia wielowyrazowe ze spacjami
    '''
    return "https://en.wikipedia.org/wiki/" + word.strip().replace(" ", "_")



In [84]:
def search(iter, links, searchWord, searchTables, searchSectionNames, searchReferences, linksFromTables):
    '''
    Funkcja przeszukująca pojedynczą stronę na wikipedii pod kątem szukanego słowa oraz dodająca do podanego słownika linki do innych artykułów
    na wikipedii znalezionych na stronie.
    W przypadku znalezienia szukanego słowa, zwraca jego odległość od wyjściowego hasła.
    Aktualizuje słownik z linkami w miejscu.
    '''
    pageKey = list(links.keys())[iter]
    resp = requests.get(pageKey)
    page = resp.text
    soup = BeautifulSoup(page, 'html.parser').css.select("#bodyContent")[0] #zawężenie wszystkich przyszłych wyszukiwań do elementów artykulu, nie całej strony
    fullText = getPlainTextFromParagraphs(soup) #dodaje do tymczasowej zmiennej tekst z paragrafów
    if searchSectionNames:
        fullText += getPlainTextFromSectionNames(soup) #dodaje do tymczasowej zmiennej tekst z nazw Sekcji
    if searchReferences:
        fullText += getPlainTextFromReferences(soup) #dodaje do tymczasowej zmiennej tekst z sekcji źródeł
    if searchTables == "yes":
        fullText += getPlainTextFromTables(soup) #dodaje do tymczasowej zmiennej tekst z tabel
    elif searchTables == "infobox_only":
        fullText += getPlainTextFromInfoboxOnly(soup) #dodaje do tymczasowej zmiennej tekst z infoboxów

    if checkSubstring(searchWord, fullText): #sprawdza, czy w zebranych danych tekstowych ze strony występuje szukane słowo
        return links[pageKey] 

    getLinksFromParagraphs(soup, links, pageKey) #dodaje do słownika linków linki z paragrafów wraz z ich odległościami od słowa wyjściowego
    if linksFromTables == "yes":
        getLinksFromTables(soup, links, pageKey) #dodaje do słownika linków linki z tabel wraz z ich odległościami od słowa wyjściowego
    elif linksFromTables == "infobox_only":
        getLinksFromInfoboxOnly(soup, links, pageKey) #dodaje do słownika linków linki z infoboxów wraz z ich odległościami od słowa wyjściowego

    return False
        
    
    

In [85]:
#Funkcje do pobierania tekstu ze strony

def getPlainTextFromParagraphs(soup):
    fullText = ""
    for i in soup.css.select("p"):
        fullText += i.getText()
    return fullText

def getPlainTextFromSectionNames(soup):
    fullText = ""
    for i in soup.css.select("div.mw-heading"):
        fullText += i.getText().replace("[edit]", "")
    return fullText

def getPlainTextFromReferences(soup):
    fullText = ""
    for i in soup.css.select("cite"):
        fullText += i.getText()
    return fullText

def getPlainTextFromTables(soup):
    fullText = ""
    for i in soup.css.select("table"):
        fullText += i.getText()
    return fullText

def getPlainTextFromInfoboxOnly(soup):
    fullText = ""
    for i in soup.css.select("table"):
        if i.get("class") is not None and "infobox" in i.get("class"):
            fullText += i.getText()
    return fullText

In [86]:
#Funkcje do pobierania linków ze strony

def getLinksFromParagraphs(soup, links, pageKey):
    for i in soup.css.select("p"):
        for j in i.find_all("a"):
            if j.get("href").startswith("/wiki/") and "https://en.wikipedia.org" + j.get("href") not in links:
                    links["https://en.wikipedia.org" + j.get("href")] = links[pageKey] + 1
    return links

def getLinksFromTables(soup, links, iter):
    for i in soup.css.select("tables"):
        for j in i.find_all("a"):
            if j.get("href").startswith("/wiki/") and "https://en.wikipedia.org" + j.get("href") not in links:
                    links["https://en.wikipedia.org" + j.get("href")] = links[pageKey] + 1
    return links

def getLinksFromInfoboxOnly(soup, links, iter):
    for i in soup.css.select("tables"):
        if i.get("class") is not None and "infobox" in i.get("class"):
            for j in i.find_all("a"):
                if j.get("href").startswith("/wiki/") and "https://en.wikipedia.org" + j.get("href") not in links:
                        links["https://en.wikipedia.org" + j.get("href")] = links[pageKey] + 1
    return links

In [87]:
def checkSubstring(word, fullText):
    '''
    Funkcja sprawdzająca wystąpienia substringa word w stringu fullText z uwzględnieniem czy słowo word występuje w zbiorze słów fullText w sensie językowym
    '''
    word = word.lower()
    fullText = fullText.lower()
    n = len(fullText)
    pattern = re.compile("[^a-zA-Z0-9]") #regex do sprawdzania, czy znaki bezpośrednio poprzedzające i następujące po znalezionym substringu są literami lub cyframi

    for m in re.finditer(word, fullText):
        if ((m.start() == 0) or (m.start() > 0 and pattern.match(fullText[m.start() - 1]))) and ((m.end() == n) or (m.end() < n and pattern.match(fullText[m.end()]))):
            return True #warunek "if(): true; else: false" ze względu na działanie funkcji re.match() 
    return False

### Przykład działania

In [88]:
starttime = time.process_time()
dist = calculateDistance("Dmitri_Sannikov", "russian")
endtime = time.process_time()
print("Distance: " + str(dist))
print("process_time: " + str(endtime - starttime))

Distance: 1
process_time: 0.234375


In [89]:
starttime = time.process_time()
dist = calculateDistance("Dmitri_Sannikov", "soccer")
endtime = time.process_time()
print("Distance: " + str(dist))
print("process_time: " + str(endtime - starttime))

Distance: 2
process_time: 2.234375


In [90]:
starttime = time.process_time()
dist = calculateDistance("Dmi_Sann", "russian")
endtime = time.process_time()
print("Distance: " + str(dist))
print("process_time: " + str(endtime - starttime))

Distance: No wikipedia page for provided word
process_time: 0.109375
